<a href="https://colab.research.google.com/github/SridharSola/Knowledge-Distillation-FER/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def adjust_learning_rate(optimizer): 
  for param_group in optimizer.param_groups: 
      param_group["lr"] /= 10.

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

def training(teacher, student, classifier, train_set, test_set, learning_rate, epochs, lamda, log, model_save, class_save):
  if torch.cuda.is_available():
    teacher.cuda()
    student.cuda()
    classifier.cuda()
  teacher.eval()
  student.train()
  classifier.train()
  optimizer = torch.optim.SGD([{'params':student.parameters(), 'lr' : learning_rate, 'momentum': 0.9, 'weight_decay': 0.00005},
                               {'params' : classifier.parameters(), 'lr' : learning_rate, 'momentum' : 0.9, 'weight_decay': 0.00005}
                                ])
  logfile = open(log, 'w')
  n_epochs = epochs
  print_every = 5
  test_loss_min = np.Inf
  test_loss_u = []
  test_loss_m = []
  
  test_acc_emoU = []
  test_acc_emoM =[]
  train_loss = []
  train_acc = []
  Ubest_test_acc = 0.0
  Mbest_test_acc = 0.0
  total_step = len(train_set)
  lrs = []
  lrs.append(learning_rate)


  #Losses:
  Lce = nn.CrossEntropyLoss().cuda()
  Lmse = nn.MSELoss().cuda()
  ce = nn.CrossEntropyLoss().cuda()

  print('\nTraining starting:\n', file = logfile)
  print('\nTraining starting:\n')
  for epoch in range(1, n_epochs+1):
      running_loss = 0.0
      correct = 0
      total = 0
      print(f'Epoch {epoch}\n', file = logfile)
      print(f'Epoch {epoch}\n')
      if epoch == 20 or epoch == 28 or epoch==36 or epoch == 50 or epoch == 60:     
          #scheduler.step()
          adjust_learning_rate(optimizer)
          lrs.append(optimizer.param_groups[0]["lr"])
          print(f'Updated lr: {lrs[-1]}\n', file = logfile)
          print(f'Updated lr: {lrs[-1]}\n')
      for batch_idx, (data_teach, data_stu, target_) in enumerate(train_set):
        optimizer.zero_grad()
        teacher_vector = teacher(data_teach)
        student_vector = student(data_stu)
        emo_probs = classifier(student_vector)

        mse = Lmse(student_vector, teacher_vector)
        ce = Lce(emo_probs, target_)
        loss = ce + lamda*mse
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, emo_pred = torch.max(emo_probs, dim = 1)
        correct += torch.sum(emo_pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
          print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch, n_epochs, batch_idx, total_step, loss.item()), file = logfile)
          print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
      train_acc.append(100 * correct / total)
      train_loss.append(running_loss/total_step)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}', file = logfile)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
      u_batch_loss = 0
      m_batch_loss = 0
      total_t=0
      u_emo_correct_t=0
      m_emo_correct_t=0
      with torch.no_grad():
          student.eval()
          classifier.eval()
          for udata_t, mdata_t, target_t in (test_set):
              udata_t, mdata_t, target_t = udata_t.to(device), mdata_t.to(device), target_t.to(device)
              
              u_v = student(udata_t)
              m_v = student(mdata_t)
              
              u_emo_probs_t = classifier(u_v)
              m_emo_probs_t = classifier(m_v)

              _,u_emo_pred_t = torch.max(u_emo_probs_t, dim=1)
              #u_mask_pred_t = torch.round(u_mask_prob_t)
              #u_mask_correct_t += torch.sum(u_mask_pred_t==target_u).item()
              u_emo_correct_t += torch.sum(u_emo_pred_t==target_t).item()

              _,m_emo_pred_t = torch.max(m_emo_probs_t, dim=1)
              #m_mask_pred_t = torch.round(m_mask_prob_t)
              #m_mask_correct_t += torch.sum(m_mask_pred_t== target_m).item()
              m_emo_correct_t += torch.sum(m_emo_pred_t==target_t).item()

              total_t += target_t.size(0)
          #test_acc_maskU.append(100 * u_mask_correct_t/total_t)
          test_acc_emoU.append(100 * u_emo_correct_t/total_t)
          test_acc_emoM.append(100 * m_emo_correct_t/total_t)

          network_learned = Ubest_test_acc < test_acc_emoU[-1] or Mbest_test_acc < test_acc_emoM[-1]
          
          print(f'Unmasked Test Accuracies: \n FER: {(100 * u_emo_correct_t/total_t):.4f}', file = logfile)  
          
          print(f'Unmasked Test Accuracies: \n FER: {(100 * u_emo_correct_t/total_t):.4f}')
          
         
          print(f'Masked Test Accuracies: \n FER: {(100 * m_emo_correct_t/total_t):.4f}', file = logfile)  
          
          print(f'Masekd Test Accuracies: \n FER: {(100 * m_emo_correct_t/total_t):.4f}')
        
          if Ubest_test_acc < test_acc_emoU[-1]:
            #Updating the best test_accuracy obtained
            print("Best FER Test Accuracy Updated for unmasked set", file = logfile)
            print("Best FER Test Accuracy Updated for unmasked set")
            Ubest_test_acc = test_acc_emoU[-1]
     
          if Mbest_test_acc < test_acc_emoM[-1]:
            #Updating the best test_accuracy obtained
            print("Best FER Test Accuracy Updated for masked set", file = logfile)
            print("Best FER Test Accuracy Updated for masked set")
            Mbest_test_acc = test_acc_emoM[-1]
          
          if network_learned:
              u_test_loss_min = u_batch_loss
              torch.save(student.state_dict(), model_save)
              torch.save(classifier.state_dict(), class_save)
              print('Improvement-Detected for unmasked, save-model to drive', file = logfile)
              print('Improvement-Detected for unmasked, save-model to drive')
          

      student.train()
      classifier.train()

  print("The best test accuracy obtained(Unmasked): ", Ubest_test_acc)
  print("The best test accuracy obtained(Masked): ", Mbest_test_acc)
  #End of training

def training_teacher(net, cls, train_set, test_set, criterion, learning_rate, epochs, log, model_save, class_save):
  if torch.cuda.is_available():
    net.cuda()
    cls.cuda()
  criterion = criterion
  optimizer = torch.optim.Adam([{'params':net.parameters(), 'lr' : learning_rate, 'momentum': 0.9, 'weight_decay': 0.00005},
                               {'params' : cls.parameters(), 'lr' : learning_rate, 'momentum' : 0.9, 'weight_decay': 0.00005}
                                ])
  logfile = open(log, 'w')
  n_epochs = epochs
  print_every = 5
  test_loss_min = np.Inf
  test_loss = []
  test_acc = []
  train_loss = []
  train_acc = []
  best_test_acc = 0.0
  total_step = len(train_set)
  lrs = []
  lrs.append(learning_rate)
  print('\nTraining starting:\n', file = logfile)
  print('\nTraining starting:\n')
  for epoch in range(1, n_epochs+1):
      running_loss = 0.0
      correct = 0
      total=0
      print(f'Epoch {epoch}\n', file = logfile)
      print(f'Epoch {epoch}\n')
      if epoch == 20 or epoch == 28 or epoch==36:     
          #scheduler.step()
          adjust_learning_rate(optimizer)
          lrs.append(optimizer.param_groups[0]["lr"])
          print(f'Updated lr: {lrs[-1]}\n', file = logfile)
          print(f'Updated lr: {lrs[-1]}\n')
      for batch_idx, (data_, target_) in enumerate(train_set):
          data_, target_ = data_.to(device), target_.to(device)
          optimizer.zero_grad()
          
          outputsV = net(data_)
          outputs = cls(outputsV)
          loss = criterion(outputs, target_)
          loss.backward()
          optimizer.step()
          #out, probs = cls(outputs)
          
          running_loss += loss.item()
          _,pred = torch.max(outputs, dim=1)
          correct += torch.sum(pred==target_).item()
          total += target_.size(0)
          if (batch_idx) % 20 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch, n_epochs, batch_idx, total_step, loss.item()), file = logfile)
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
      train_acc.append(100 * correct / total)
      train_loss.append(running_loss/total_step)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}', file = logfile)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
      batch_loss = 0
      total_t=0
      correct_t=0
      
      with torch.no_grad():
          net.eval()
          cls.eval()
          for data_t, target_t in (test_set):
              data_t, target_t = data_t.to(device), target_t.to(device)
              outputs_tV = net(data_t)
              outputs_t = cls(outputs_tV)
              #outputs_t, probs_t = cls(outputs_t1)
              loss_t = criterion(outputs_t, target_t)
              batch_loss += loss_t.item()
              _,pred_t = torch.max(outputs_t, dim=1)
              correct_t += torch.sum(pred_t==target_t).item()
              total_t += target_t.size(0)
          test_acc.append(100 * correct_t/total_t)
          test_loss.append(batch_loss/len(test_loader))
          network_learned = batch_loss < test_loss_min
          print(f'test loss: {np.mean(test_loss):.4f}, test acc: {(100 * correct_t/total_t):.4f}\n', file = logfile)
          print(f'test loss: {np.mean(test_loss):.4f}, test acc: {(100 * correct_t/total_t):.4f}\n')

          if best_test_acc < test_acc[-1]:
            #Updating the best test_accuracy obtained
            print("Best Test Accuracy Updated", file = logfile)
            print("Best Test Accuracy Updated")
            best_test_acc = test_acc[-1]
          if network_learned:
              test_loss_min = batch_loss
              torch.save(net.state_dict(), model_save)
              torch.save(cls.state_dict(), class_save)
              print('Improvement-Detected, save-model to drive', file = logfile)
              print('Improvement-Detected, save-model to drive')


      net.train()
      cls.train()

  print("The best test accuracy obtained was: ", best_test_acc)
  #End of training